In [ ]:
import netCDF4 as nc
import scipy 
import os
import re
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask
from xmhw.xmhw import threshold, detect
from datetime import date
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.express as px
import plotly.graph_objects as go
import hashlib
import matplotlib.colors as mcolors

In [ ]:
input_location = 'Kalbarri'
input_lat= -27.5
input_lon = 114.3

In [ ]:
df_site_kelp = pd.read_csv("kalbarri_abundance.csv") 
# Sort values
df_site_kelp = df_site_kelp.sort_values(by=['survey_year'])
# Convert to datetime format
df_site_kelp['survey_datetime'] = pd.to_datetime(df_site_kelp['survey_year'].astype(str) + '-01-01')
df_site_kelp = df_site_kelp.groupby(['survey_year']).mean().reset_index(level=['survey_year'])

df_site_kelp['location'] = input_location
df_site_kelp['latitude'] = -27.5
df_site_kelp['longitude'] = 114.3
df_site_kelp.head()


In [ ]:
# Group by site_name and calculate rolling differences
df_site_kelp['survey_change'] = df_site_kelp['survey_mean'].diff()

df_site_kelp= df_site_kelp[(df_site_kelp.survey_year<2014) & (df_site_kelp.survey_year!=2006)]
df_site_kelp

In [ ]:
def summer_mhw_metric(input_location):
    infile = 'OBIS_kelp_sst.nc'    
    
    ds = xr.open_dataset(infile).sel(
        time = slice('1991-01-01','2024-12-31')).sel(
        lat = slice(input_lat-0.25, input_lat+0.25), lon = slice(input_lon-0.25, input_lon+0.25)).squeeze('zlev', drop=True).mean(dim=("lat", "lon"))
    ds_summer = ds.sel(time=ds.sst.time.dt.month.isin([12,1,2,3,4]))
    
    date = np.array(ds_summer.time,dtype='datetime64[ns]')
    df = pd.DataFrame(date, columns=['date']) 
    df['doy'] = df['date'].dt.dayofyear
    df['temp'] = pd.DataFrame(ds_summer.sst.values)
    t = ds.time.values
    sst = ds.sst.values
    clim = threshold(ds.sst, climatologyPeriod=[1991, 2020]) # threshold and climatology
    maxthresh = clim.seas.max().values 
    
    # Convert the xarray DataArray to a DataFrame
    climatology_df = clim.to_dataframe().reset_index()

    # Merge the two DataFrames on the 'doy' column
    merged_df = pd.merge(df, climatology_df, on='doy', how='left')  

    # add the dhd column, only keep positive intense values as our dhd
    merged_df['dhd50'] = (merged_df.temp - maxthresh).where((merged_df.temp - maxthresh)>0, 0)  
    merged_df['max_inten'] = merged_df.temp - maxthresh   ### intensity relative to different climatology values
    ## ==========================temp, cum_inten, max_inten ===========================================
    summer_mhw_metrix = merged_df[['temp', 'dhd50', 'max_inten']].set_index(merged_df.date)
    summer_mhw_metrix.columns = ['summer_temp', 'summer_dhd50', 'summer_inten']
    # get mean summer temp and sum of summer intense
    summer_mhw_metrix = summer_mhw_metrix.resample('AS-DEC').agg({'summer_temp': 'mean', 'summer_dhd50': 'sum', 'summer_inten': 'max'})
    # add one year
    summer_mhw_metrix['survey_year'] = summer_mhw_metrix.index.year+1
    
    ## ============================= dt dt =========================================================
    df = df.set_index('date')
    dt = 10
    dTdt = pd.DataFrame()
    for i in df.index.year.unique()[0:-1]:
        ssti = df[(df.index.year == i) & (df.index.month > 11) | (df.index.year == i+1) & (df.index.month <5)]
        
        # calculate n-day jumping mean
        jmeani = ssti.resample('10D').mean()

        # calculate dT as difference between consecutive 10-day means
        dT = jmeani.diff(periods=1)
        dTdt = pd.concat([dTdt, dT/dt])  #.mean()
        dTdt = dTdt.dropna()

    summer_dtdt = dTdt.resample('AS-DEC').max()
    summer_dtdt = summer_dtdt.rename(columns={'temp': 'summer_dtdt'}) 
    summer_dtdt['survey_year'] = summer_dtdt.index.year + 1
    
    summer_mhw_metrix = summer_mhw_metrix.merge(summer_dtdt, how='inner', on='survey_year')
    
    return summer_mhw_metrix

In [ ]:
# check the output metric df
summer_mhw_metrix = summer_mhw_metric(input_location).fillna(0)
summer_mhw_metrix #.head()

In [ ]:
# Initialize a new column in df_site_kelp to store the computed max summer temperatures
df_site_kelp['summer_temp_max'] = np.nan
df_site_kelp['summer_dhd50_max'] = np.nan
df_site_kelp['summer_inten_max'] = np.nan
df_site_kelp['summer_dtdt_max'] = np.nan

for j in range(1, len(df_site_kelp.survey_year)):             # survey_year in dfi is not continuous, so
    # find the max temp between the two survey years. (j-1)th is not just one-year before jth year , it is the previous survey year before jth survey year;   
    # +1 is due to 'range' function
    summer_temp_max = summer_mhw_metrix.summer_temp[summer_mhw_metrix.survey_year.isin(range(df_site_kelp.survey_year.iloc[j-1]+1, df_site_kelp.survey_year.iloc[j]+1))].max() 
    summer_dhd50_max = summer_mhw_metrix.summer_dhd50[summer_mhw_metrix.survey_year.isin(range(df_site_kelp.survey_year.iloc[j-1]+1, df_site_kelp.survey_year.iloc[j]+1))].max()
    summer_inten_max = summer_mhw_metrix.summer_inten[summer_mhw_metrix.survey_year.isin(range(df_site_kelp.survey_year.iloc[j-1]+1, df_site_kelp.survey_year.iloc[j]+1))].max()
    summer_dtdt_max = summer_mhw_metrix.summer_dtdt[summer_mhw_metrix.survey_year.isin(range(df_site_kelp.survey_year.iloc[j-1]+1, df_site_kelp.survey_year.iloc[j]+1))].max() 

    # Update the summer_temp_max value in the corresponding row of df_site_kelp
    mask = (df_site_kelp.survey_year == df_site_kelp.survey_year.iloc[j])
    df_site_kelp.loc[mask, 'summer_temp_max'] = summer_temp_max
    df_site_kelp.loc[mask, 'summer_dhd50_max'] = summer_dhd50_max
    df_site_kelp.loc[mask, 'summer_inten_max'] = summer_inten_max
    df_site_kelp.loc[mask, 'summer_dtdt_max'] = summer_dtdt_max

# Now df_site_kelp has an additional column 'summer_temp_max' with the computed max summer temperatures
df_site_kelp = df_site_kelp.dropna()
df_site_kelp

In [ ]:
## ======= saving processed data ==========
## Change commenting lines# df_site_kelp.to_csv('4pop_change_rawdata.csv', index=False)
df_existing = pd.read_csv('4pop_change_rawdata.csv')
df_combined = pd.concat([df_existing, df_site_kelp], ignore_index=True) 
df_combined.to_csv('4pop_change_rawdata.csv', index=False)

In [ ]:
df_kalbarri = df_site_kelp.copy()

#### Fitting a model for summer_temp!

In [ ]:
# Add a constant (i.e., bias or intercept) to the independent variable
x = df_kalbarri.summer_temp_max

# This is your dependent variable
y = df_kalbarri.survey_change

x_with_const = sm.add_constant(x)


In [ ]:
# Create the OLS model
model = sm.OLS(y, x_with_const).fit()

x_vals = np.linspace(x.min(), x.max(), 100)
x_vals_constant = sm.add_constant(x_vals)
y_pred = model.predict(x_vals_constant)
# Get predictions
predictions = model.get_prediction(x_vals_constant)
# Extract the mean predictions and the confidence intervals
# pred_means = predictions.predicted_mean
pred_cis = predictions.conf_int()

model.summary()

In [ ]:
# Plot the original data points
plt.scatter(x, y, color='blue', label='Original Data')

# Plot the regression line
plt.plot(x_vals, y_pred, color='red', label='Fitted Line')

# Plot the confidence intervals
plt.fill_between(x_vals, pred_cis[:, 0], pred_cis[:, 1], color='red', alpha=0.2, label='95% CI')

In [ ]:
df_model = pd.DataFrame({"location": 'Kalbarri',
                         "summer_temp_max": x_vals,
                         "predicted_change": y_pred,
                         'lower_bound':  pred_cis[:, 0], 
                         'upper_bound': pred_cis[:, 1],
                        })

In [ ]:
## ======= saving processed data ==========
df_existing = pd.read_csv('model_change_relationship.csv')
df_combined = pd.concat([df_existing, df_model], ignore_index=True)
# Save the combined data back to CSV
df_combined.to_csv('model_change_relationship.csv', index=False)

#### Fitting a model for dhd!

In [ ]:
# Add a constant (i.e., bias or intercept) to the independent variable
x = df_site_kelp.summer_dhd50_max
x_with_const = sm.add_constant(x)

# This is your dependent variable
y = df_site_kelp.survey_change

In [ ]:
# Create the OLS model
model = sm.OLS(y, x_with_const).fit()

x_vals = np.linspace(x.min(), x.max(), 500)
x_vals_constant = sm.add_constant(x_vals)
y_pred = model.predict(x_vals_constant)
# Get predictions
predictions = model.get_prediction(x_vals_constant)
# Extract the mean predictions and the confidence intervals
# pred_means = predictions.predicted_mean
pred_cis = predictions.conf_int()

model.summary()

In [ ]:
# Plot the original data points
plt.scatter(x, y, color='blue', label='Original Data')

# Plot the regression line
plt.plot(x_vals, y_pred, color='red', label='Fitted Line')

# Plot the confidence intervals
plt.fill_between(x_vals, pred_cis[:, 0], pred_cis[:, 1], color='red', alpha=0.2, label='95% CI')

In [ ]:
df_model = pd.DataFrame({"location": 'Kalbarri',
                         "summer_dhd50_max": x_vals,
                         "predicted_change": y_pred,
                         'lower_bound':  pred_cis[:, 0], 
                         'upper_bound': pred_cis[:, 1],
                        })


In [ ]:
## ======= saving processed data ==========
df_existing = pd.read_csv('dhdmodel_change_relationship.csv')
df_combined = pd.concat([df_existing, df_model], ignore_index=True)
# Save the combined data back to CSV
df_combined.to_csv('dhdmodel_change_relationship.csv', index=False)

#### Fitting a model for max_intensity

In [ ]:
# Add a constant (i.e., bias or intercept) to the independent variable
x = df_site_kelp.summer_inten_max
x_with_const = sm.add_constant(x)

# This is your dependent variable
y = df_site_kelp.survey_change

In [ ]:
# Create the OLS model
model = sm.OLS(y, x_with_const).fit()

x_vals = np.linspace(x.min(), x.max(), 500)
x_vals_constant = sm.add_constant(x_vals)
y_pred = model.predict(x_vals_constant)
# Get predictions
predictions = model.get_prediction(x_vals_constant)
# Extract the mean predictions and the confidence intervals
# pred_means = predictions.predicted_mean
pred_cis = predictions.conf_int()

model.summary()

In [ ]:
# Plot the original data points
plt.scatter(x, y, color='blue', label='Original Data')

# Plot the regression line
plt.plot(x_vals, y_pred, color='red', label='Fitted Line')

# Plot the confidence intervals
plt.fill_between(x_vals, pred_cis[:, 0], pred_cis[:, 1], color='red', alpha=0.2, label='95% CI')

In [ ]:
df_model = pd.DataFrame({"location": 'Kalbarri',
                         "summer_inten_max": x_vals,
                         "predicted_change": y_pred,
                         'lower_bound':  pred_cis[:, 0], 
                         'upper_bound': pred_cis[:, 1],
                        })


In [ ]:
df_existing = pd.read_csv('inten_max_model_change_relationship.csv')
df_combined = pd.concat([df_existing, df_model], ignore_index=True)
# Save the combined data back to CSV
df_combined.to_csv('inten_max_model_change_relationship.csv', index=False)

#### Fitting a model for dtdt

In [ ]:
# Add a constant (i.e., bias or intercept) to the independent variable
x = df_site_kelp.summer_dtdt_max
x_with_const = sm.add_constant(x)

# This is your dependent variable
y = df_site_kelp.survey_change

In [ ]:
# Create the OLS model
model = sm.OLS(y, x_with_const).fit()

x_vals = np.linspace(x.min(), x.max(), 100)
x_vals_constant = sm.add_constant(x_vals)
y_pred = model.predict(x_vals_constant)
# Get predictions
predictions = model.get_prediction(x_vals_constant)
# Extract the mean predictions and the confidence intervals
# pred_means = predictions.predicted_mean
pred_cis = predictions.conf_int()

model.summary()

In [ ]:
# Plot the original data points
plt.scatter(x, y, color='blue', label='Original Data')

# Plot the regression line
plt.plot(x_vals, y_pred, color='red', label='Fitted Line')

# Plot the confidence intervals
plt.fill_between(x_vals, pred_cis[:, 0], pred_cis[:, 1], color='red', alpha=0.2, label='95% CI')

In [ ]:
df_model = pd.DataFrame({"location": 'Kalbarri',
                         "summer_dtdt_max": x_vals,
                         "predicted_change": y_pred,
                         'lower_bound':  pred_cis[:, 0], 
                         'upper_bound': pred_cis[:, 1],
                        })


In [ ]:
## ======= saving processed data ==========
df_existing = pd.read_csv('dtdtmodel_change_relationship.csv')
df_combined = pd.concat([df_existing, df_model], ignore_index=True)
# Save the combined data back to CSV
df_combined.to_csv('dtdtmodel_change_relationship.csv', index=False)